<a href="https://colab.research.google.com/github/EUtrilla2002/ProyectoDatos/blob/main/ProyectoDatos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Análisis de datos

In [ ]:
import zipfile
import os

zip_path = "/content/archive.zip"  # Reemplaza con el nombre de tu archivo ZIP
extract_path = "/content/contenido_zip"  # Carpeta donde se extraerán los archivos

# Crear carpeta si no existe
os.makedirs(extract_path, exist_ok=True)

# Descomprimir el ZIP
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Descompresión completa.")

## Bases de datos

In [ ]:
import pandas as pd

# Cargar los datasets (pueden estar en CSV, Excel, etc.)
df1 = pd.read_csv("/content/contenido_zip/Books_rating.csv")  # Primera base de datos
df2 = pd.read_csv("/content/contenido_zip/books_data.csv")  # Segunda base de datos

# Ver los primeros registros (opcional)
print(df1.head())
print(df2.head())


In [ ]:
# Unir las bases de datos por la columna "Título"
df_final = pd.merge(df1, df2, on="Título", how="inner")  # "inner" solo mantiene coincidencias

# Guardar el resultado en un nuevo archivo CSV
df_final.to_csv("base_combinada.csv", index=False)

print("Bases de datos unidas correctamente.")
print(df_final.head())  # Mostrar resultado

## Eliminación de columnas

In [ ]:
# Contar NaN en cada columna
nan_counts = df_final.isna().sum()

# Mostrar resultado
print(nan_counts)

In [ ]:
df_final_sincolumnas = df_final.drop(columns=["NombreColumna"])